In [1]:
# import math
# from nltk.tokenize import RegexpTokenizer
# from nltk.stem import WordNetLemmatizer
# wordnet_lemmatizer = WordNetLemmatizer()
# import numpy as np
# import pandas as pd


import math
import json
# import numpy  as np
import nltk
nltk.download('stopwords')
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
from nltk.corpus import stopwords 
from nltk.tokenize import RegexpTokenizer

Vocab={}
n_gramlist_fequency={}
starting_word_of_sentence={}
def smoothening(n_gramlist_fequency):
    sz_Vocab=len(Vocab)
    all_smoothen={}
    laPlace_smoothing_unigram={}
    laPlace_smoothing_bigram={}
    laPlace_smoothing_trigram={}
    val=sum(n_gramlist_fequency['unigram'].values())
    for x,y in n_gramlist_fequency['unigram'].items():
#         value = ((n_gramlist_fequency['unigram'][x]+1)/(val+sz_Vocab))
        value = ((y+1)/(val+sz_Vocab))
        laPlace_smoothing_unigram[x]=math.log10(value)
    all_smoothen['unigram']=laPlace_smoothing_unigram
    
    for x1,y1 in n_gramlist_fequency['bigram'].items():
        ew1=x1.split()
        key1 = ew1[1]+'|'+ew1[0]
        value1=((y1+1)/(n_gramlist_fequency['unigram'][ew1[0]]+sz_Vocab))
        laPlace_smoothing_bigram[key1]=math.log10(value1)
    all_smoothen['bigram']=laPlace_smoothing_bigram
    for x2,y2 in n_gramlist_fequency['trigram'].items():
        ew2=x2.split()
        key2 = ew2[2]+'|'+ew2[0]+' '+ew2[1]
        value2=((y2+1)/(n_gramlist_fequency['bigram'][ew2[0]+' '+ew2[1]]+sz_Vocab))
        laPlace_smoothing_trigram[key2]=math.log10(value2)
    all_smoothen['trigram']=laPlace_smoothing_trigram    
    return all_smoothen
def create_n_igramlist(list,n):
    unigram=list
    bigram=[]
    trigram=[]
    for k in range(len(unigram)-1):
        bigram.append(unigram[k]+' '+unigram[k+1])
    for j in range(len(unigram)-2):
        trigram.append(unigram[j]+' '+unigram[j+1]+' '+unigram[j+2])
    ngrams={}
    ngrams['unigram']=unigram
    ngrams['bigram']=bigram
    ngrams['trigram']=trigram
    return ngrams
def frequency_of_words_in_list(ls):
    freq={}
    for i in ls:
        if i in freq:
            freq[i]+=1
        else:
            freq[i]=1
    return freq
def tokeninfolderfile(comment_in_class):
    l=[]
    n_gramlist={}
    global Vocab
    n_gramlist_fequency={}
    n_gramlist_fequency['unigram']={}
    n_gramlist_fequency['bigram']={}
    n_gramlist_fequency['trigram']={}
    
    n_gramlist['unigram']=[]
    n_gramlist['bigram']=[]
    n_gramlist['trigram']=[]
    
    for comment in comment_in_class:        
        tk = RegexpTokenizer(r'[\w]+[-\']?[\w]*')   
        newtokens=tk.tokenize(comment)
        new_words=[] 
        for w in newtokens:
                w=wordnet_lemmatizer.lemmatize(w.lower())
                new_words.append(w)   
                if w in Vocab:
                    Vocab[w]+=1
                else:
                    Vocab[w]=1                        
        n_g_list=create_n_igramlist(new_words,0)
        n_gramlist['unigram'].extend(n_g_list['unigram'])
        n_gramlist['bigram'].extend(n_g_list['bigram'])
        n_gramlist['trigram'].extend(n_g_list['trigram'])
            
    n_gramlist_fequency['unigram']=frequency_of_words_in_list(n_gramlist['unigram'])
#     print(len(n_gramlist_fequency['unigram']))
#     print(len(Vocab))
    n_gramlist_fequency['bigram']=frequency_of_words_in_list(n_gramlist['bigram'])
    n_gramlist_fequency['trigram']=frequency_of_words_in_list(n_gramlist['trigram'])   
#         l.extend(new_words)
#     return l
    return n_gramlist_fequency

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
import pandas as pd
data = pd.read_csv('jigsaw-toxic-comment-classification-challenge/train.csv')
cmnt_text = list(data['comment_text'])
toxic = list(data['toxic'])
st = list(data['severe_toxic'])
ob = list(data['obscene'])
th = list(data['threat'])
ins = list(data['insult'])
ih = list(data['identity_hate'])
toxic_comments = []
severe_toxic_comments = []
obscene_comments = []
threat_comments = []
insult_comments = []
identity_hate_comments = []

for i in range(0,len(toxic)):
    if(toxic[i] == 1):
        toxic_comments+=[cmnt_text[i]]
        
for i in range(0,len(st)):
    if(st[i] == 1):
        severe_toxic_comments+=[cmnt_text[i]]
        
for i in range(0,len(ob)):
    if(ob[i] == 1):
        obscene_comments+=[cmnt_text[i]]
        
for i in range(0,len(th)):
    if(th[i] == 1):
        threat_comments+=[cmnt_text[i]]

for i in range(0,len(ins)):
    if(ins[i] == 1):
        insult_comments+=[cmnt_text[i]]
        
for i in range(0,len(ih)):
    if(ih[i] == 1):
        identity_hate_comments+=[cmnt_text[i]]
dict_comments = {"toxic":toxic_comments,"severe_toxic":severe_toxic_comments, "obscene":obscene_comments, "threat":threat_comments, "insult":insult_comments, "identity_hate":identity_hate_comments }


In [0]:
f_per_word_per_class={}
for i,j in dict_comments.items():
    f_per_word_per_class[i]={}
    f_per_word_per_class[i]=tokeninfolderfile(j)
    
# print(f_per_word_per_class['toxic'])

In [0]:
smothen={}
for i,j in f_per_word_per_class.items():
    smothen[i]={}
    smothen[i]=smoothening(j)

In [0]:
def sentence_probability_per_class(f_per_word_per_class,smothen,yoursentence):
        global Vocab
        tk = RegexpTokenizer(r'[\w]+[-\']?[\w]*')
        # yoursentence=input("Enter a valid sentence\n")
        # yoursentence="abcdbcbcbcbbcbcbbc cbcbbcbbcbbcbbc bcbcbcbbcbbcbcbjshksdkaskd cjakcjskjhckjsc When I got my knee rebuilt I got back on the street bike ASAP. I put the crutches on the rack and the passenger seat and they hung out back a LONG way. Just make sure they're tied down tight in front and no problemo."
#         yoursentence="fuck you ash hole mother fucker you dick is too dusty"
        newtokens=tk.tokenize(yoursentence)
        new_words=[]
        for w in newtokens:
                w=wordnet_lemmatizer.lemmatize(w.lower())   
                new_words.append(w)
        n_g_list=create_n_igramlist(new_words,0)
        prob_of_sent_per_class_per_gram={}
        for toxic_class,toxic_class_data in smothen.items():
            prob_of_sent_per_class_per_gram[toxic_class]={}
            for n_gram,n_gram_values in toxic_class_data.items():
                if(n_gram=="unigram"):
                    Unigram_prob_of_sen=0
                    for w in n_g_list['unigram']:
                        if(w in smothen[toxic_class]['unigram']):
                            Unigram_prob_of_sen=Unigram_prob_of_sen+smothen[toxic_class]['unigram'][w]
                        else:
                            Unigram_prob_of_sen=Unigram_prob_of_sen+(math.log10(1/(sum(f_per_word_per_class[toxic_class]['unigram'].values())+(len(smothen[toxic_class]['unigram'])))))  
                    prob_of_sent_per_class_per_gram[toxic_class][n_gram]=Unigram_prob_of_sen
                elif(n_gram=="bigram"):
                    Bigram_prob_of_sen=0
                    for w in n_g_list['bigram']:
                        split_word=w.split()
                        key = split_word[1]+'|'+split_word[0]
                        sz_Vocab=len(Vocab)
                    #     print(sz_Vocab)
                        if(key in smothen[toxic_class]['bigram']):
                            Bigram_prob_of_sen=Bigram_prob_of_sen+smothen[toxic_class]['bigram'][key]
                        else:
                            if(split_word[0] in f_per_word_per_class[toxic_class]['unigram'].keys()):
                                unigrampresent=f_per_word_per_class[toxic_class]['unigram'][split_word[0]]
                            else:
                                unigrampresent=0
                            Bigram_prob_of_sen=Bigram_prob_of_sen+math.log10(1/(unigrampresent+sz_Vocab))
                    prob_of_sent_per_class_per_gram[toxic_class][n_gram]=Bigram_prob_of_sen

                elif(n_gram=="trigram"):
                    trigram_prob_of_sen=0
                    for w in n_g_list['trigram']:
                        split_word=w.split()
                        key = split_word[2]+'|'+split_word[0]+' '+split_word[1]
                        if(key in smothen[toxic_class]['trigram']):
                    #         print(des_laPlace_smoothing_trigram[key])
                            trigram_prob_of_sen=trigram_prob_of_sen+smothen[toxic_class]['trigram'][key]
                        else:
                            prev_bigramword=split_word[0]+' '+split_word[1]
                            if(prev_bigramword in f_per_word_per_class[toxic_class]['bigram'].keys()):
                                pre_bigrampresent=f_per_word_per_class[toxic_class]['bigram'][prev_bigramword]
                            else:
                                pre_bigrampresent=0
                            trigram_prob_of_sen=trigram_prob_of_sen+math.log10(1/(pre_bigrampresent+sz_Vocab))
                    prob_of_sent_per_class_per_gram[toxic_class][n_gram]=trigram_prob_of_sen
        return prob_of_sent_per_class_per_gram
            
prob_per_class_sent={}
yoursentence="fuck you ash hole mother fucker you dick is too dusty"
prob_per_class_sent=sentence_probability_per_class(f_per_word_per_class,smothen,yoursentence)
# print(prob_per_class_sent)
# for toxic_class,toxic_class_data in prob_per_class_sent.items():
#     print(toxic_class)
#     for n_gram,n_gram_values in toxic_class_data.items():
#         print(n_gram,n_gram_values)
#     print("\n")

In [0]:
# #read text data
# data = pd.read_csv('jigsaw-toxic-comment-classification-challenge/test.csv')
# # cmnt_text = list(data['comment_text'])
# per_comment_prob={}
# for y,x in enumerate(list(data['comment_text'])):
#     per_comment_prob[data['id'][y]]=sentence_probability_per_class(f_per_word_per_class,smothen,x)
        
# #     per_comment_prob[]sentence_probability_per_class(f_per_word_per_class,smothen,x)
# # per_comment_prob   

In [0]:
# import pickle
# def storeData(filename,datais):
#     dbfile = open(filename, 'ab')
#     pickle.dump(datais, dbfile)
#     dbfile.close()   
# def loadData(filename):
#     dbfile = open(filename, 'rb')
#     datais=pickle.load(dbfile)
#     dbfile.close()
#     return datais;
# storeData('test_data_of_all',per_comment_prob)    
# print("done")
# # print(loadData('test_data_of_1000'))
# # print(len(per_comment_prob))

In [0]:
import pickle
def storeData(filename,datais):
    dbfile = open(filename, 'ab')
    pickle.dump(datais, dbfile)
    dbfile.close()

    
def loadData(filename):
    dbfile = open(filename, 'rb')
    datais=pickle.load(dbfile)
    dbfile.close()
    return datais
abc={}
abc=loadData('test_data_of_all')
k={}
label_data = pd.read_csv('jigsaw-toxic-comment-classification-challenge/test_labels.csv')
list_of_file_to_find_ac=[]
for x,y in enumerate(label_data['id']):
#         if(x<1000):
            if((label_data["toxic"][x] == 1 or label_data["severe_toxic"][x] == 1 or  label_data["obscene"][x] == 1 or label_data["threat"][x] == 1 or label_data["insult"][x] == 1 or label_data["identity_hate"][x] == 1 )):
                list_of_file_to_find_ac.append(y)

uni_max_val=-9223372036854775808
bi_max_val=-9223372036854775808
tri_max_val=-9223372036854775808
max_uni=''
max_bi=''
max_tri=''
sentence_belong_to_class={}
for x,y in abc.items():
#     print(y)
    if(x in list_of_file_to_find_ac):
        pqr={}
        for toxic_class,toxic_class_data in y.items():
            if(toxic_class_data['unigram']>uni_max_val):
                uni_max_val=toxic_class_data['unigram']
                max_uni=toxic_class
            if(toxic_class_data['bigram']>bi_max_val):
                bi_max_val=toxic_class_data['bigram']
                max_bi=toxic_class
            if(toxic_class_data['trigram']>tri_max_val):
                tri_max_val=toxic_class_data['trigram']
                max_tri=toxic_class
        pqr['unigram']=max_uni
        pqr['bigram']=max_bi
        pqr['trigram']=max_tri
        sentence_belong_to_class[x]={}
        sentence_belong_to_class[x]=pqr
# print(len(sentence_belong_to_class))

In [0]:
list_of_class=["toxic","severe_toxic","obscene","threat","insult","identity_hate"]
unigram_class_index_all_files=[]
bigram_class_index_all_files=[]
trigram_class_index_all_files=[]
for x,y in sentence_belong_to_class.items():
    unigram_class_index_all_files.append(list_of_class.index(y['unigram']))
    bigram_class_index_all_files.append(list_of_class.index(y['bigram']))
    trigram_class_index_all_files.append(list_of_class.index(y['trigram']))
# print(unigram_class_index_all_files)
# print(bigram_class_index_all_files)
# print(trigram_class_index_all_files)

In [0]:
list_of_class=["toxic","severe_toxic","obscene","threat","insult","identity_hate"]
import pandas as pd
label_data = pd.read_csv('jigsaw-toxic-comment-classification-challenge/test_labels.csv')
orignal_class_index_all_files=[]
for x,y in enumerate(label_data['id']):
#     if(x<1000):
        if(label_data["toxic"][x] == 1):
            orignal_class_index_all_files.append(list_of_class.index("toxic"))
        elif(label_data["severe_toxic"][x] == 1):
            orignal_class_index_all_files.append(list_of_class.index("severe_toxic"))
        elif(label_data["obscene"][x] == 1):
            orignal_class_index_all_files.append(list_of_class.index("obscene"))
        elif(label_data["threat"][x] == 1):
            orignal_class_index_all_files.append(list_of_class.index("threat"))
        elif(label_data["insult"][x] == 1):
            orignal_class_index_all_files.append(list_of_class.index("insult"))
        elif(label_data["identity_hate"][x] == 1):
            orignal_class_index_all_files.append(list_of_class.index("identity_hate"))
        
#     else:
#         break

In [0]:
#unigram_acuracy
# print(len(orignal_class_index_all_files))
# print(len(unigram_class_index_all_files))
# print(len(bigram_class_index_all_files))
# print(len(trigram_class_index_all_files))
from sklearn.metrics import accuracy_score
print("accuracy_score of unigram ",accuracy_score(orignal_class_index_all_files,unigram_class_index_all_files))
print("accuracy_score of bigram ",accuracy_score(orignal_class_index_all_files,bigram_class_index_all_files))
print("accuracy_score of trigram ",accuracy_score(orignal_class_index_all_files,trigram_class_index_all_files))

accuracy_score of unigram  0.15361204549094987
accuracy_score of bigram  0.9532276149287202
accuracy_score of trigram  0.9753323722569277
